In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()  # take environment variables from .env.
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [3]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content
    return {"joke": response}

def generate_explanation(state: JokeState):
    prompt = f'Explain the following joke: {state["joke"]}'
    response = llm.invoke(prompt).content
    return {"explanation": response}

In [5]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [6]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({"topic": "burgers"}, config=config1)

{'topic': 'burgers',
 'joke': 'Why did the burger go to the gym?\n\nBecause it wanted to get better buns! 🍔💪',
 'explanation': 'This joke plays on a double meaning of the word "buns." In the context of a burger, "buns" refers to the bread that holds the burger together. However, "buns" is also a colloquial term for a person\'s buttocks, often associated with fitness and body shape. \n\nThe humor comes from the unexpected twist: the burger, a food item, is anthropomorphized as if it has personal goals and desires, like wanting to improve its physical appearance. The punchline suggests that the burger wants to "get better buns" in both senses—having tastier bread and, humorously, a better-looking backside. The combination of food and fitness creates a lighthearted and playful image, making the joke amusing.'}

In [7]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'burgers', 'joke': 'Why did the burger go to the gym?\n\nBecause it wanted to get better buns! 🍔💪', 'explanation': 'This joke plays on a double meaning of the word "buns." In the context of a burger, "buns" refers to the bread that holds the burger together. However, "buns" is also a colloquial term for a person\'s buttocks, often associated with fitness and body shape. \n\nThe humor comes from the unexpected twist: the burger, a food item, is anthropomorphized as if it has personal goals and desires, like wanting to improve its physical appearance. The punchline suggests that the burger wants to "get better buns" in both senses—having tastier bread and, humorously, a better-looking backside. The combination of food and fitness creates a lighthearted and playful image, making the joke amusing.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a420d-b8fb-6d84-8002-c784fbbe1ac9'}}, metadata={'source': 'loop', '

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'burgers', 'joke': 'Why did the burger go to the gym?\n\nBecause it wanted to get better buns! 🍔💪', 'explanation': 'This joke plays on a double meaning of the word "buns." In the context of a burger, "buns" refers to the bread that holds the burger together. However, "buns" is also a colloquial term for a person\'s buttocks, often associated with fitness and body shape. \n\nThe humor comes from the unexpected twist: the burger, a food item, is anthropomorphized as if it has personal goals and desires, like wanting to improve its physical appearance. The punchline suggests that the burger wants to "get better buns" in both senses—having tastier bread and, humorously, a better-looking backside. The combination of food and fitness creates a lighthearted and playful image, making the joke amusing.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a420d-b8fb-6d84-8002-c784fbbe1ac9'}}, metadata={'source': 'loop', 

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({"topic": "programming"}, config=config2)

{'topic': 'programming',
 'joke': 'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!',
 'explanation': 'This joke plays on a double meaning of the word "bugs." \n\nIn the context of programming, "bugs" refer to errors or flaws in software code that can cause it to behave unexpectedly. Programmers often spend a lot of time debugging their code to find and fix these issues.\n\nOn the other hand, in a more literal sense, "bugs" can refer to insects, which are often attracted to light. This is a common observation; for example, you might see moths flying around a light bulb at night.\n\nThe humor comes from the clever wordplay: programmers prefer dark mode (a display setting with a dark background and light text) because, in a humorous twist, it suggests that using dark mode helps them avoid attracting "bugs" (both the software errors and the insects). It\'s a light-hearted way to highlight a common preference among programmers while also making a pun.'}

In [11]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'programming', 'joke': 'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!', 'explanation': 'This joke plays on a double meaning of the word "bugs." \n\nIn the context of programming, "bugs" refer to errors or flaws in software code that can cause it to behave unexpectedly. Programmers often spend a lot of time debugging their code to find and fix these issues.\n\nOn the other hand, in a more literal sense, "bugs" can refer to insects, which are often attracted to light. This is a common observation; for example, you might see moths flying around a light bulb at night.\n\nThe humor comes from the clever wordplay: programmers prefer dark mode (a display setting with a dark background and light text) because, in a humorous twist, it suggests that using dark mode helps them avoid attracting "bugs" (both the software errors and the insects). It\'s a light-hearted way to highlight a common preference among programmers while also making a pun.

In [12]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'programming', 'joke': 'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!', 'explanation': 'This joke plays on a double meaning of the word "bugs." \n\nIn the context of programming, "bugs" refer to errors or flaws in software code that can cause it to behave unexpectedly. Programmers often spend a lot of time debugging their code to find and fix these issues.\n\nOn the other hand, in a more literal sense, "bugs" can refer to insects, which are often attracted to light. This is a common observation; for example, you might see moths flying around a light bulb at night.\n\nThe humor comes from the clever wordplay: programmers prefer dark mode (a display setting with a dark background and light text) because, in a humorous twist, it suggests that using dark mode helps them avoid attracting "bugs" (both the software errors and the insects). It\'s a light-hearted way to highlight a common preference among programmers while also making a pun

### Time Travel

In [14]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0a420d-7b38-64e8-8000-7acce5019651"}})

StateSnapshot(values={'topic': 'burgers'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0a420d-7b38-64e8-8000-7acce5019651'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-10-08T08:29:01.326036+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a420d-7b32-62d2-bfff-65eec73b206b'}}, tasks=(PregelTask(id='e6c24abe-7312-8bda-553e-e08bfbb0f20d', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the burger go to the gym?\n\nBecause it wanted to get better buns! 🍔💪'}),), interrupts=())

In [15]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0a420d-7b38-64e8-8000-7acce5019651"}})  # should be instant

{'topic': 'burgers',
 'joke': 'Why did the burger go to the gym?\n\nBecause it wanted to get better buns! 🍔💪',
 'explanation': 'This joke plays on a double meaning of the word "buns." In the context of a burger, "buns" refers to the bread that holds the burger together. However, "buns" is also a colloquial term for a person\'s buttocks, often associated with fitness and body shape. \n\nThe humor comes from the idea of a burger, which is typically seen as an unhealthy food, wanting to improve its "buns" by going to the gym, as if it were a person trying to get in shape. The unexpected personification of the burger and the clever wordplay create a lighthearted and amusing punchline. The emoji adds a fun visual element, reinforcing the fitness theme.'}